<a href="https://colab.research.google.com/github/Panachai-spec007/dataviz2025/blob/main/student_health_warehouse_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Lab: สร้าง Health Data Warehouse (ฉบับนักศึกษา)

**ชื่อ**: ___________________________  **รหัสนักศึกษา**: _______________

## 🎯 วัตถุประสงค์:
ฝึกปฏิบัติการสร้าง Data Warehouse จากฐานข้อมูลระบบเฝ้าระวังโรค NCDs

## 📚 สิ่งที่จะได้เรียนรู้:
1. การเชื่อมต่อและดึงข้อมูลจาก SQLite Database
2. การทำ Data Transformation (JOIN, Groupby, Merge)
3. การสร้าง Data Warehouse 5 ตาราง
4. การอัปโหลดข้อมูลไปยัง Google Sheets

## ⏱️ เวลาที่ใช้: 120 นาที

---

## 📦 Part 1: เตรียมสภาพแวดล้อม (10 นาที)

### Exercise 1.1: ติดตั้ง Libraries

**โจทย์**: ติดตั้ง packages ที่จำเป็นสำหรับ Lab นี้

**Hint**: ต้องติดตั้ง `gspread`, `google-auth`, `google-auth-oauthlib`, `google-auth-httplib2`, `openpyxl`

In [1]:
# TODO: ติดตั้ง packages ที่จำเป็น
# Hint: ใช้ !pip install ... -q

# เขียนโค้ดที่นี่
!pip install gspread oauth2client openpyxl google-auth-oauthlib google-auth-httplib2 google-auth -q

print("✅ ติดตั้ง libraries สำเร็จ")

✅ ติดตั้ง libraries สำเร็จ


### Exercise 1.2: Mount Google Drive

**โจทย์**: Mount Google Drive และตรวจสอบว่ามีไฟล์ที่ต้องการหรือไม่

**ไฟล์ที่ต้องการ**:
- `ncds_sample.db`
- `districts_with_zone_20250930_084409.xlsx`

**Path**: `/content/drive/MyDrive/Dashboard-Teaching-2025/`

In [2]:
# TODO: Mount Google Drive
# Hint: ใช้ from google.colab import drive
#       drive.mount('/content/drive')

# เขียนโค้ดที่นี่

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# TODO: กำหนด path ของไฟล์
import os

SHARED_FOLDER = '/content/drive/MyDrive/Dataviz2025/SHARED_FOLDER'
DB_PATH = os.path.join(SHARED_FOLDER, 'ncds_sample.db')
DISTRICTS_PATH = os.path.join(SHARED_FOLDER, 'districts_with_zone_20250930_084409.xlsx')

# TODO: ตรวจสอบว่าไฟล์มีอยู่จริง
# Hint: ใช้ os.path.exists()

# เขียนโค้ดที่นี่
if os.path.exists(DB_PATH) and os.path.exists(DISTRICTS_PATH):
    print("ทำถูกแล้วพ่อหนุ่ม")
else:
    print("ให้ทายว่าถูกไหม")

ทำถูกแล้วพ่อหนุ่ม


### Exercise 1.3: Import Libraries

**โจทย์**: Import libraries ที่จำเป็นทั้งหมด

In [4]:
# TODO: Import libraries ที่จำเป็น
# Hint: sqlite3, pandas, numpy, datetime, gspread, warnings

# เขียนโค้ดที่นี่
import sqlite3
import pandas as pd
import numpy as np
import datetime
import gspread
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Import libraries สำเร็จ")

✅ Import libraries สำเร็จ


---

## 🔌 Part 2: เชื่อมต่อและดึงข้อมูล (15 นาที)

### Exercise 2.1: เชื่อมต่อ SQLite Database

**โจทย์**: เชื่อมต่อกับ `ncds_sample.db` และแสดงรายชื่อตารางทั้งหมดพร้อมจำนวนแถว

In [5]:
# TODO: เชื่อมต่อ SQLite Database
# Hint: ใช้ sqlite3.connect(DB_PATH)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# TODO: ดูรายชื่อตารางทั้งหมดพร้อมจำนวนแถว
# Hint: SELECT name FROM sqlite_master WHERE type='table'
#       จากนั้น SELECT COUNT(*) FROM {table_name}

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"Table: {table_name}, Rows: {count}")



Table: campaign, Rows: 20
Table: ncds_profile, Rows: 192
Table: ncds_record, Rows: 192
Table: ncds_habit, Rows: 192
Table: memberdata, Rows: 1920


### Exercise 2.2: ดึงข้อมูลจากตาราง

**โจทย์**: ดึงข้อมูลจาก 5 ตารางหลัก:
1. `ncds_profile`
2. `ncds_record`
3. `ncds_habit`
4. `campaign`
5. `memberdata`

**Hint**: ใช้ `pd.read_sql_query("SELECT * FROM table_name", conn)`

In [6]:
# TODO: ดึงข้อมูลจากแต่ละตาราง

print("📥 กำลังดึงข้อมูลจาก Database...")

# 1. ncds_profile
ncds_profile = pd.read_sql_query("SELECT * FROM ncds_profile", conn)
print(f"✅ ncds_profile: {len(ncds_profile)} แถว")

# 2. ncds_record
ncds_record = pd.read_sql_query("SELECT * FROM ncds_record", conn)
print(f"✅ ncds_record: {len(ncds_record)} แถว")

# 3. ncds_habit
ncds_habit = pd.read_sql_query("SELECT * FROM ncds_habit", conn)
print(f"✅ ncds_habit: {len(ncds_habit)} แถว")

# 4. campaign
campaign = pd.read_sql_query("SELECT * FROM campaign", conn)
print(f"✅ campaign: {len(campaign)} แถว")

# 5. memberdata
memberdata = pd.read_sql_query("SELECT * FROM memberdata", conn)
print(f"✅ memberdata: {len(memberdata)} แถว")

📥 กำลังดึงข้อมูลจาก Database...
✅ ncds_profile: 192 แถว
✅ ncds_record: 192 แถว
✅ ncds_habit: 192 แถว
✅ campaign: 20 แถว
✅ memberdata: 1920 แถว


### Exercise 2.3: โหลดข้อมูลพื้นที่

**โจทย์**: โหลดข้อมูลพื้นที่จากไฟล์ Excel และแปลง `district_code` เป็น string

In [7]:
# TODO: โหลดข้อมูลพื้นที่
# Hint: ใช้ pd.read_excel()

districts_df = pd.read_excel(DISTRICTS_PATH)
districts_df['district_code'] = districts_df['district_code'].astype(str)

print(f"✅ โหลดข้อมูลพื้นที่: {len(districts_df)} แถว")
display(districts_df.head())

✅ โหลดข้อมูลพื้นที่: 7356 แถว


,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat,lng,zone
0,100101,แขวง พระบรมมหาราชวัง,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.492,4.0
1,100102,แขวง วังบูรพาภิรมย์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.744,100.499,4.0
2,100103,แขวง วัดราชบพิธ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.750,100.499,4.0
3,100104,แขวง สำราญราษฎร์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.503,4.0
4,100105,แขวง ศาลเจ้าพ่อเสือ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.754,100.497,4.0


---

## 🏗️ Part 3: สร้าง Data Warehouse (40 นาที)

### Exercise 3.1: Health Profile Warehouse

**โจทย์**: สร้างตาราง Health Profile Warehouse ตามขั้นตอนต่อไปนี้:

**ขั้นตอน**:
1. ดึงข้อมูล**ล่าสุด**ของแต่ละสมาชิกจาก `ncds_profile` และ `ncds_record`
2. นับจำนวนครั้งการตรวจของแต่ละสมาชิก
3. JOIN กับ `campaign` เพื่อได้ `district_code`
4. JOIN กับ `districts_df` เพื่อได้ข้อมูลพื้นที่
5. สร้าง DataFrame ตามคอลัมน์ที่กำหนด

**คอลัมน์ที่ต้องมี**:
- member_id
- จำนวนครั้งการตรวจ
- เพศ (แปลง 0→หญิง, 1→ชาย)
- อายุ
- ตำบล_id, ตำบล
- อำเภอ_id, อำเภอ
- จังหวัด_id, จังหวัด
- เขต
- กลุ่มอายุ
- ประวัติการเจ็บป่วย (code และ label)
- ประวัติแพ้ยา (code และ label)
- ประวัติแพ้อาหาร (code และ label)
- ญาติสายตรงNCDs (code และ label)

In [8]:
# Step 1: ดึงข้อมูลล่าสุดของแต่ละสมาชิก
latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)
screening_count = ncds_record.groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

# JOIN ข้อมูล
campaign['district_code'] = campaign['district_code'].astype(str)
profile_campaign = pd.merge(latest_profiles, campaign[['id', 'district_code']],
                            left_on='campaignId', right_on='id', how='left')
profile_location = pd.merge(profile_campaign, districts_df, on='district_code', how='left')
profile_record = pd.merge(profile_location,
                         latest_records[['memberid', 'drug_allergy', 'food_allergy', 'record_ncds', 'relatives_ncds']],
                         on='memberid', how='left')
final_data = pd.merge(profile_record, screening_count, on='memberid', how='left')
final_data['จำนวนครั้งการตรวจ'] = final_data['จำนวนครั้งการตรวจ'].fillna(0).astype(int)

# Helper functions
def handle_text_data(value):
    return 999 if pd.isna(value) or value == '' else str(value)

def handle_numeric_data(value):
    return 999 if pd.isna(value) else float(value)

def get_gender_label(gender):
    if pd.isna(gender): return 999
    return "หญิง" if int(gender) == 0 else "ชาย"

def get_age_group(age):
    if pd.isna(age) or age == 999 or age == 0: return 999
    if age < 20: return 'อายุต่ำกว่า 20 ปี'
    if age < 35: return 'อายุระหว่าง 20-34 ปี'
    if age < 60: return 'อายุระหว่าง 35-59 ปี'
    return 'อายุ 60 ปีขึ้นไป'

def convert_value(value, labels):
    """แปลงค่า -1, 1 เป็น label"""
    if pd.isna(value) or value == '' or value == 999: return labels[0]
    if value == -1: return labels[1]
    if value == 1: return labels[2]
    return labels[0]

# สร้าง Warehouse
health_profile_wh = pd.DataFrame({
    'member_id': final_data['memberid'],
    'จำนวนครั้งการตรวจ': final_data['จำนวนครั้งการตรวจ'],
    'เพศ': final_data['gender'].apply(get_gender_label),
    'อายุ': final_data['age'].apply(handle_numeric_data),
    'ตำบล_id': final_data['district_code'].apply(handle_text_data),
    'ตำบล': final_data['district_name'].apply(handle_text_data),
    'อำเภอ_id': final_data['amphur_code'].apply(handle_numeric_data),
    'อำเภอ': final_data['amphur_name'].apply(handle_text_data),
    'จังหวัด_id': final_data['province_code'].apply(handle_numeric_data),
    'จังหวัด': final_data['province_name'].apply(handle_text_data),
    'เขต': final_data['zone'].apply(handle_numeric_data),
    'กลุ่มอายุ': final_data['age'].apply(get_age_group),
    'ประวัติการเจ็บป่วย_code': final_data['record_ncds'].apply(handle_numeric_data),
    'ประวัติการเจ็บป่วย': final_data['record_ncds'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการเจ็บป่วยรวม'])),
    'ประวัติแพ้ยา_code': final_data['drug_allergy'].apply(handle_numeric_data),
    'ประวัติแพ้ยา': final_data['drug_allergy'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการแพ้ยา'])),
    'ประวัติแพ้อาหาร_code': final_data['food_allergy'].apply(handle_numeric_data),
    'ประวัติแพ้อาหาร': final_data['food_allergy'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการแพ้อาหาร'])),
    'ญาติสายตรงNCDs_code': final_data['relatives_ncds'].apply(handle_numeric_data),
    'ญาติสายตรงNCDs': final_data['relatives_ncds'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการเจ็บป่วยรวม']))
})

print(f"✅ Health Profile Warehouse: {len(health_profile_wh)} แถว")
health_profile_wh.head()

✅ Health Profile Warehouse: 100 แถว


,member_id,จำนวนครั้งการตรวจ,เพศ,อายุ,ตำบล_id,ตำบล,อำเภอ_id,อำเภอ,จังหวัด_id,จังหวัด,เขต,กลุ่มอายุ,ประวัติการเจ็บป่วย_code,ประวัติการเจ็บป่วย,ประวัติแพ้ยา_code,ประวัติแพ้ยา,ประวัติแพ้อาหาร_code,ประวัติแพ้อาหาร,ญาติสายตรงNCDs_code,ญาติสายตรงNCDs
0,10000036,1,ชาย,50.0,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,1.0,มีประวัติการแพ้ยา,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
1,10000058,2,หญิง,44.0,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,6.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
2,10000025,1,หญิง,73.0,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,1.0,อายุ 60 ปีขึ้นไป,-1.0,ไม่มีประวัติ,1.0,มีประวัติการแพ้ยา,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
3,10000040,1,หญิง,48.0,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,1.0,มีประวัติการเจ็บป่วยรวม
4,10000076,1,หญิง,62.0,100101,แขวง พระบรมมหาราชวัง,1001.0,เขต พระนคร,10.0,กรุงเทพมหานคร,4.0,อายุ 60 ปีขึ้นไป,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ


In [10]:
# Step 2: นับจำนวนครั้งการตรวจ
# Hint: ใช้ .groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

screening_count = ncds_record.groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

print(f"✓ Screening count: {len(screening_count)} สมาชิก")
screening_count.head()

✓ Screening count: 100 สมาชิก


,memberid,จำนวนครั้งการตรวจ
0,10000000,3
1,10000001,3
2,10000002,3
3,10000003,2
4,10000004,2


In [12]:
# Step 3: JOIN กับ campaign
# Hint: ใช้ pd.merge() โดย left_on='campaignId', right_on='id'

campaign['district_code'] = campaign['district_code'].astype(str)

profile_campaign = pd.merge(latest_profiles, campaign[['id', 'district_code']],
                            left_on='campaignId', right_on='id', how='left')

print(f"✓ Profile + Campaign: {len(profile_campaign)} แถว")

✓ Profile + Campaign: 100 แถว


In [14]:
# Step 4: JOIN กับ districts
# Hint: ใช้ pd.merge() โดย on='district_code'

profile_location = pd.merge(profile_campaign, districts_df, on='district_code', how='left')

print(f"✓ Profile + Location: {len(profile_location)} แถว")

✓ Profile + Location: 100 แถว


In [16]:
# Step 5: JOIN กับ record และ screening_count
# Hint: JOIN 2 ครั้ง - ครั้งแรกกับ latest_records, ครั้งที่สองกับ screening_count

profile_record = pd.merge(profile_location,
                         latest_records[['memberid', 'drug_allergy', 'food_allergy', 'record_ncds', 'relatives_ncds']],
                         on='memberid', how='left')

final_data = pd.merge(profile_record, screening_count, on='memberid', how='left')

# แปลง NaN ในจำนวนครั้งการตรวจเป็น 0
final_data['จำนวนครั้งการตรวจ'] = final_data['จำนวนครั้งการตรวจ'].fillna(0).astype(int)

print(f"✓ Final data: {len(final_data)} แถว")
final_data.head()

✓ Final data: 100 แถว


,id_x,dateadd,detail,memberid,age,gender,campaignId,risk_bmi,risk_habit,risk_waistline,risk_bloodpressure,risk_fbs,risk_movement,risk_sleep,risk_2q,risk_stress,risk_vegetable,risk_sugar,risk_fat,risk_sodium,id_y,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat,lng,zone,drug_allergy,food_allergy,record_ncds,relatives_ncds,จำนวนครั้งการตรวจ
0,77,2025-06-02 00:00:00,0b654fa141f37aee35abf9d3555527b94bf7b9b2e167dc...,10000036,50,1,100008,low,medium,medium,high,,high,high,,medium,high,high,,medium,100008,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,18.796,98.985,2.0,1,-1,-1,-1,1
1,114,2025-06-10 00:00:00,da3b431e84ad23aecc05a3a6ca86c0aa3ee563a21e6b23...,10000058,44,0,100006,high,medium,low,high,low,low,low,,low,low,medium,medium,high,100006,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,13.850,100.509,6.0,-1,-1,-1,-1,2
2,54,2025-06-12 00:00:00,ffd144bbca0915dedc29ff24a4990a2a4a08ed615f3b98...,10000025,73,0,100015,high,medium,high,low,medium,medium,low,high,medium,medium,medium,low,low,100015,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,16.438,102.837,1.0,1,-1,-1,-1,1
3,82,2025-06-12 00:00:00,a33f5d57dd594b547ab361798301edb6a7075de5f9496a...,10000040,48,0,100008,low,low,,low,high,high,high,low,low,high,medium,high,high,100008,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,18.796,98.985,2.0,-1,-1,-1,1,1
4,142,2025-06-13 00:00:00,2ccc87026f02cff661c222f46039fb18cbc46d0900fce4...,10000076,62,0,100001,medium,low,medium,low,high,,low,medium,high,,low,high,,100001,100101,แขวง พระบรมมหาราชวัง,1001.0,เขต พระนคร,10.0,กรุงเทพมหานคร,13.751,100.492,4.0,-1,-1,-1,-1,1


In [19]:
# Step 6: สร้าง Helper Functions
# TODO: เขียนฟังก์ชันแปลงค่าต่างๆ

def get_gender_label(gender):
    """แปลงเพศ: 0→หญิง, 1→ชาย, อื่นๆ→999"""
    if pd.isna(gender): return 999
    return "หญิง" if int(gender) == 0 else "ชาย"

def get_age_group(age):
    """
    แปลงกลุ่มอายุ:
    - < 20: 'อายุต่ำกว่า 20 ปี'
    - 20-34: 'อายุระหว่าง 20-34 ปี'
    - 35-59: 'อายุระหว่าง 35-59 ปี'
    - >= 60: 'อายุ 60 ปีขึ้นไป'
    """
    if pd.isna(age) or age == 999 or age == 0: return 999
    if age < 20: return 'อายุต่ำกว่า 20 ปี'
    if age < 35: return 'อายุระหว่าง 20-34 ปี'
    if age < 60: return 'อายุระหว่าง 35-59 ปี'
    return 'อายุ 60 ปีขึ้นไป'

def convert_value(value, labels):
    """แปลงค่า -1, 1 เป็น label"""
    if pd.isna(value) or value == '' or value == 999: return labels[0]
    if value == -1: return labels[1]
    if value == 1: return labels[2]
    return labels[0]

def convert_illness_history(value):
    """
    แปลงประวัติการเจ็บป่วย:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการเจ็บป่วยรวม'
    - อื่นๆ: 'ไม่ทราบ'
    """
    return convert_value(value, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการเจ็บป่วยรวม'])


# TODO: เขียนฟังก์ชันเพิ่มเติมสำหรับ drug_allergy, food_allergy, relatives_ncds
# (รูปแบบเหมือน convert_illness_history)

def convert_drug_allergy(value):
    """
    แปลงประวัติแพ้ยา:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการแพ้ยา'
    - อื่นๆ: 'ไม่ทราบ'
    """
    return convert_value(value, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการแพ้ยา'])

def convert_food_allergy(value):
    """
    แปลงประวัติแพ้อาหาร:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการแพ้อาหาร'
    - อื่นๆ: 'ไม่ทราบ'
    """
    return convert_value(value, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการแพ้อาหาร'])

def convert_relatives_ncds(value):
    """
    แปลงญาติสายตรง NCDs:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการเจ็บป่วยรวม'
    - อื่นๆ: 'ไม่ทราบ'
    """
    return convert_value(value, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการเจ็บป่วยรวม'])

In [20]:
# Step 7: สร้าง Health Profile Warehouse
# TODO: สร้าง DataFrame ตามคอลัมน์ที่กำหนด

health_profile_wh = pd.DataFrame({
    'member_id': final_data['memberid'],
    'จำนวนครั้งการตรวจ': final_data['จำนวนครั้งการตรวจ'],
    'เพศ': final_data['gender'].apply(get_gender_label),
    'อายุ': final_data['age'], # Assuming age is already numeric, adjust if needed
    'ตำบล_id': final_data['district_code'].apply(lambda x: str(x) if pd.notna(x) else '999'), # Ensure string type
    'ตำบล': final_data['district_name'].apply(lambda x: str(x) if pd.notna(x) else 'ไม่ทราบ'),
    'อำเภอ_id': final_data['amphur_code'].apply(lambda x: str(int(x)) if pd.notna(x) else '999'), # Ensure string type
    'อำเภอ': final_data['amphur_name'].apply(lambda x: str(x) if pd.notna(x) else 'ไม่ทราบ'),
    'จังหวัด_id': final_data['province_code'].apply(lambda x: str(int(x)) if pd.notna(x) else '999'), # Ensure string type
    'จังหวัด': final_data['province_name'].apply(lambda x: str(x) if pd.notna(x) else 'ไม่ทราบ'),
    'เขต': final_data['zone'].apply(lambda x: float(x) if pd.notna(x) else 999),
    'กลุ่มอายุ': final_data['age'].apply(get_age_group),
    'ประวัติการเจ็บป่วย_code': final_data['record_ncds'].apply(lambda x: float(x) if pd.notna(x) else 999),
    'ประวัติการเจ็บป่วย': final_data['record_ncds'].apply(convert_illness_history),
    'ประวัติแพ้ยา_code': final_data['drug_allergy'].apply(lambda x: float(x) if pd.notna(x) else 999),
    'ประวัติแพ้ยา': final_data['drug_allergy'].apply(convert_drug_allergy),
    'ประวัติแพ้อาหาร_code': final_data['food_allergy'].apply(lambda x: float(x) if pd.notna(x) else 999),
    'ประวัติแพ้อาหาร': final_data['food_allergy'].apply(convert_food_allergy),
    'ญาติสายตรงNCDs_code': final_data['relatives_ncds'].apply(lambda x: float(x) if pd.notna(x) else 999),
    'ญาติสายตรงNCDs': final_data['relatives_ncds'].apply(convert_relatives_ncds)
})

print(f"✅ Health Profile Warehouse: {len(health_profile_wh)} แถว")
health_profile_wh.head()

✅ Health Profile Warehouse: 100 แถว


,member_id,จำนวนครั้งการตรวจ,เพศ,อายุ,ตำบล_id,ตำบล,อำเภอ_id,อำเภอ,จังหวัด_id,จังหวัด,เขต,กลุ่มอายุ,ประวัติการเจ็บป่วย_code,ประวัติการเจ็บป่วย,ประวัติแพ้ยา_code,ประวัติแพ้ยา,ประวัติแพ้อาหาร_code,ประวัติแพ้อาหาร,ญาติสายตรงNCDs_code,ญาติสายตรงNCDs
0,10000036,1,ชาย,50,500101,ต. ศรีภูมิ,5001,อ. เมืองเชียงใหม่,50,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,1.0,มีประวัติการแพ้ยา,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
1,10000058,2,หญิง,44,120102,ต. ตลาดขวัญ,1201,อ. เมืองนนทบุรี,12,จ. นนทบุรี,6.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
2,10000025,1,หญิง,73,400101,ต. ในเมือง,4001,อ. เมืองขอนแก่น,40,จ. ขอนแก่น,1.0,อายุ 60 ปีขึ้นไป,-1.0,ไม่มีประวัติ,1.0,มีประวัติการแพ้ยา,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
3,10000040,1,หญิง,48,500101,ต. ศรีภูมิ,5001,อ. เมืองเชียงใหม่,50,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,1.0,มีประวัติการเจ็บป่วยรวม
4,10000076,1,หญิง,62,100101,แขวง พระบรมมหาราชวัง,1001,เขต พระนคร,10,กรุงเทพมหานคร,4.0,อายุ 60 ปีขึ้นไป,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ


### Exercise 3.2: NCDs Disease Warehouse

**โจทย์**: สร้างตารางประวัติโรค NCDs

**ขั้นตอน**:
1. ดึงข้อมูลล่าสุดจาก `ncds_record`
2. สำหรับแต่ละสมาชิก สร้างรายการโรคของผู้ตรวจ (8 โรค)
3. สร้างรายการโรคของญาติ (9 โรค)
4. รวมเป็นตารางเดียว โดยแต่ละแถว = 1 โรค

**โรคของผู้ตรวจ**: cancer, chronic_kidney_disease, cirrhosis, dementia, diabetes, hyperlipidemia, hypertension, stroke

**โรคของญาติ**: cancer, cirrhosis, coronary_artery_disease, dementia, diabetes, hyperlipidemia, hypertension, stroke, obesity

In [17]:
# TODO: สร้าง NCDs Disease Warehouse
# Hint: ดูตัวอย่างจากไฟล์ detailed_health_warehouse_tutorial.ipynb

# เขียนโค้ดที่นี่
# ดึงข้อมูลล่าสุด
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)

# Mapping โรค
personal_diseases = {
    'record_cancer': 'โรคมะเร็ง',
    'record_chronic_kidney_disease': 'โรคไตเรื้อรัง',
    'record_cirrhosis': 'โรคตับแข็ง',
    'record_dementia': 'โรคสมองเสื่อม',
    'record_diabetes': 'โรคเบาหวาน',
    'record_hyperlipidemia': 'โรคไขมันในเลือด',
    'record_hypertension': 'โรคความดันโลหิตสูง',
    'record_stroke': 'โรคหลอดเลือดสมอง'
}

relatives_diseases = {
    'relatives_cancer': 'โรคมะเร็ง',
    'relatives_cirrhosis': 'โรคตับแข็ง',
    'relatives_coronary_artery_disease': 'โรคหลอดเลือดหัวใจ',
    'relatives_dementia': 'โรคสมองเสื่อม',
    'relatives_diabetes': 'โรคเบาหวาน',
    'relatives_hyperlipidemia': 'โรคไขมันในเลือด',
    'relatives_hypertension': 'โรคความดันโลหิตสูง',
    'relatives_stroke': 'โรคหลอดเลือดสมอง',
    'relatives_obesity': 'โรคอ้วนลงพุง'
}

# สร้างรายการโรค
disease_records = []
for _, row in latest_records.iterrows():
    member_id = row['memberid']

    # โรคของผู้ตรวจ
    p_diseases = [{'code': k, 'name': v} for k, v in personal_diseases.items()
                  if k in row and pd.notna(row[k]) and row[k] == 1]

    # โรคของญาติ
    r_diseases = [{'code': k, 'name': v} for k, v in relatives_diseases.items()
                  if k in row and pd.notna(row[k]) and row[k] == 1]

    max_len = max(len(p_diseases), len(r_diseases), 1)

    for i in range(max_len):
        disease_records.append({
            'member_id': member_id,
            'no': i + 1 if i < len(p_diseases) else 999,
            'โรค_code': p_diseases[i]['code'] if i < len(p_diseases) else 999,
            'โรคของผู้ตรวจ': p_diseases[i]['name'] if i < len(p_diseases) else 999,
            'no_โรคญาติผู้ตรวจ': i + 1 if i < len(r_diseases) else 999,
            'โรคญาติผู้ตรวจ_code': r_diseases[i]['code'] if i < len(r_diseases) else 999,
            'โรคของญาติผู้ตรวจ': r_diseases[i]['name'] if i < len(r_diseases) else 999
        })

ncds_disease_wh = pd.DataFrame(disease_records)
print(f"✅ NCDs Disease Warehouse: {len(ncds_disease_wh)} แถว")
ncds_disease_wh.head(10)


✅ NCDs Disease Warehouse: 308 แถว


,member_id,no,โรค_code,โรคของผู้ตรวจ,no_โรคญาติผู้ตรวจ,โรคญาติผู้ตรวจ_code,โรคของญาติผู้ตรวจ
0,10000036,999,999,999,1,relatives_coronary_artery_disease,โรคหลอดเลือดหัวใจ
1,10000036,999,999,999,2,relatives_dementia,โรคสมองเสื่อม
2,10000036,999,999,999,3,relatives_stroke,โรคหลอดเลือดสมอง
3,10000058,1,record_cancer,โรคมะเร็ง,1,relatives_diabetes,โรคเบาหวาน
4,10000058,2,record_dementia,โรคสมองเสื่อม,999,999,999
5,10000058,3,record_hypertension,โรคความดันโลหิตสูง,999,999,999
6,10000058,4,record_stroke,โรคหลอดเลือดสมอง,999,999,999
7,10000025,1,record_cirrhosis,โรคตับแข็ง,1,relatives_coronary_artery_disease,โรคหลอดเลือดหัวใจ
8,10000025,2,record_hypertension,โรคความดันโลหิตสูง,2,relatives_dementia,โรคสมองเสื่อม
9,10000025,999,999,999,3,relatives_obesity,โรคอ้วนลงพุง


### Exercise 3.3: Eating Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมการกิน

**ข้อมูลที่ต้องแปลง**:
- กินผัก (vegetable): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน
- กินขนมหวาน (dessert): 0→ทุกวัน, 1→4-6 วัน, 2→1-3 วัน, 3→ไม่ทำเลย (กลับด้าน!)
- ฯลฯ

In [21]:
# TODO: สร้าง Eating Behavior Warehouse
# Hint: ต้องสร้างฟังก์ชันแปลงค่าสำหรับแต่ละพฤติกรรม

# เขียนโค้ดที่นี่
# ดึงข้อมูลล่าสุด
latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)

# ฟังก์ชันแปลงค่า
def map_score(score, mapping, reverse=False):
    if pd.isna(score) or score == -1: return 999
    score = int(score)
    return mapping.get(min(score, len(mapping)-1), mapping[len(mapping)-1])

# Mappings
freq_map = {0: "ไม่ทำเลย", 1: "1-3 วันต่อสัปดาห์", 2: "4-6 วันต่อสัปดาห์", 3: "ทุกวัน"}
freq_reverse = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
add_map = {0: "เติมทุกครั้ง", 1: "เติมบางครั้ง", 2: "ไม่เติมเลย"}

# สร้าง Warehouse
eating_behavior_wh = pd.DataFrame({
    'date': pd.to_datetime(latest_habits['dateadd']).dt.date,
    'member_id': latest_habits['memberid'],
    'กินผัก_score': latest_habits['vegetable'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินผัก': latest_habits['vegetable'].apply(lambda x: map_score(x, freq_map)),
    'กินขนมหวาน_score': latest_habits['dessert'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินขนมหวาน': latest_habits['dessert'].apply(lambda x: map_score(x, freq_reverse)),
    'ดื่มเครื่องดื่มรสหวาน_score': latest_habits['sweet_drink'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ดื่มเครื่องดื่มรสหวาน': latest_habits['sweet_drink'].apply(lambda x: map_score(x, freq_reverse)),
    'เติมน้ำตาลน้ำเชื่อม_score': latest_habits['sweet_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'เติมน้ำตาลน้ำเชื่อม': latest_habits['sweet_food'].apply(lambda x: map_score(x, add_map)),
    'อาหารไขมันสูง_score': latest_habits['fat_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'อาหารไขมันสูง': latest_habits['fat_food'].apply(lambda x: map_score(x, freq_reverse)),
    'กินอาหารแปรรูป_score': latest_habits['processed_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินอาหารแปรรูป': latest_habits['processed_food'].apply(lambda x: map_score(x, freq_reverse)),
    'กินอาหารสำเร็จรูป_score': latest_habits['instant_food'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินอาหารสำเร็จรูป': latest_habits['instant_food'].apply(lambda x: map_score(x, freq_reverse)),
    'กินอาหารปรุงรสด้วยผงปรุงรส_score': latest_habits['gourmet_powder'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'กินอาหารปรุงรสด้วยผงปรุงรส': latest_habits['gourmet_powder'].apply(lambda x: map_score(x, freq_reverse)),
    'เติมน้ำปลา_score': latest_habits['fish_sauce'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'เติมน้ำปลา': latest_habits['fish_sauce'].apply(lambda x: map_score(x, add_map))
})

print(f"✅ Eating Behavior Warehouse: {len(eating_behavior_wh)} แถว")
eating_behavior_wh.head()


✅ Eating Behavior Warehouse: 100 แถว


,date,member_id,กินผัก_score,กินผัก,กินขนมหวาน_score,กินขนมหวาน,ดื่มเครื่องดื่มรสหวาน_score,ดื่มเครื่องดื่มรสหวาน,เติมน้ำตาลน้ำเชื่อม_score,เติมน้ำตาลน้ำเชื่อม,อาหารไขมันสูง_score,อาหารไขมันสูง,กินอาหารแปรรูป_score,กินอาหารแปรรูป,กินอาหารสำเร็จรูป_score,กินอาหารสำเร็จรูป,กินอาหารปรุงรสด้วยผงปรุงรส_score,กินอาหารปรุงรสด้วยผงปรุงรส,เติมน้ำปลา_score,เติมน้ำปลา
76,2025-06-02,10000036,3,ทุกวัน,0,ทุกวัน,3,ไม่ทำเลย,1,เติมบางครั้ง,1,4-6 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,0,ทุกวัน,1,4-6 วันต่อสัปดาห์,2,ไม่เติมเลย
113,2025-06-10,10000058,999,999,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,0,เติมทุกครั้ง,0,ทุกวัน,0,ทุกวัน,3,ไม่ทำเลย,3,ไม่ทำเลย,0,เติมทุกครั้ง
53,2025-06-12,10000025,1,1-3 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,1,เติมบางครั้ง,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,2,ไม่เติมเลย
81,2025-06-12,10000040,2,4-6 วันต่อสัปดาห์,3,ไม่ทำเลย,999,999,2,ไม่เติมเลย,1,4-6 วันต่อสัปดาห์,3,ไม่ทำเลย,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,2,ไม่เติมเลย
141,2025-06-13,10000076,0,ไม่ทำเลย,999,999,999,999,1,เติมบางครั้ง,2,1-3 วันต่อสัปดาห์,1,4-6 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,3,ไม่ทำเลย,1,เติมบางครั้ง


### Exercise 3.4: Comprehensive Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมรวม

**ข้อมูลที่ต้องแปลง**:
- การเคลื่อนไหว (movement)
- การสูบบุหรี่ (smoking)
- การนอนหลับ (sleep)
- ภาวะซึมเศร้า (2q_1)
- ความเครียด (2q_2)
- การดื่มแอลกอฮอล์ (alcohol)
- การออกกำลังกาย (exercise)

In [22]:
# TODO: สร้าง Comprehensive Behavior Warehouse
# Hint: คล้ายกับ Eating Behavior แต่เป็นพฤติกรรมด้านอื่น

# เขียนโค้ดที่นี่
# Mappings
movement_map = {0: "ไม่ทำเลย", 1: "1-2 วันต่อสัปดาห์", 2: "มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์"}
smoking_map = {-1: "ไม่สูบ", 1: "สูบ"}
sleep_map = {0: "0 - 2 วันต่อสัปดาห์", 1: "3 - 5 วันต่อสัปดาห์", 2: "6 - 7 วันต่อสัปดาห์"}
depression_map = {0: "ไม่มีความเสี่ยง", 1: "มีความเสี่ยง"}
stress_map = {0: "แทบไม่มี", 1: "เป็นบางครั้ง", 2: "บ่อยครั้ง", 3: "เป็นประจำ"}
alcohol_map = {0: "ไม่ดื่มเลย", 1: "ดื่ม 1 - 3 วันต่อสัปดาห์", 2: "ดื่ม 4 - 6 วันต่อสัปดาห์", 3: "ดื่มเป็นประจำทุกวัน"}
exercise_map = {
    0: "ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก",
    1: "ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์",
    2: "ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์",
    3: "ออกกำลังกายหนัก เล่นกีฬา 6-7 วัน/สัปดาห์",
    4: "ออกกำลังกายหนักมากเป็นนักกีฬา"
}

# สร้าง Warehouse
comprehensive_behavior_wh = pd.DataFrame({
    'date': pd.to_datetime(latest_habits['dateadd']).dt.date,
    'member_id': latest_habits['memberid'],
    'การเคลื่อนไหวร่างกาย_score': latest_habits['movement'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'การเคลื่อนไหวร่างกาย': latest_habits['movement'].apply(lambda x: map_score(x, movement_map)),
    'สูบบุหรี่_code': latest_habits['smoking'].apply(lambda x: 999 if pd.isna(x) else int(x)),
    'สูบบุหรี่': latest_habits['smoking'].apply(lambda x: smoking_map.get(int(x), 999) if pd.notna(x) else 999),
    'การนอนหลับ_score': latest_habits['sleep'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'การนอนหลับ': latest_habits['sleep'].apply(lambda x: map_score(x, sleep_map)),
    'ภาวะซึมเศร้า_score': latest_habits['2q_1'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ภาวะซึมเศร้า': latest_habits['2q_1'].apply(lambda x: map_score(x, depression_map)),
    'ความเครียด_score': latest_habits['2q_2'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ความเครียด': latest_habits['2q_2'].apply(lambda x: map_score(x, stress_map)),
    'การดื่มแอล_score': latest_habits['alcohol'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'การดื่มแอล': latest_habits['alcohol'].apply(lambda x: map_score(x, alcohol_map)),
    'ออกกำลังกาย_score': latest_habits['exercise'].apply(lambda x: 999 if pd.isna(x) or x == -1 else int(x)),
    'ออกกำลังกาย': latest_habits['exercise'].apply(lambda x: map_score(x, exercise_map))
})

print(f"✅ Comprehensive Behavior Warehouse: {len(comprehensive_behavior_wh)} แถว")
comprehensive_behavior_wh.head()



✅ Comprehensive Behavior Warehouse: 100 แถว


,date,member_id,การเคลื่อนไหวร่างกาย_score,การเคลื่อนไหวร่างกาย,สูบบุหรี่_code,สูบบุหรี่,การนอนหลับ_score,การนอนหลับ,ภาวะซึมเศร้า_score,ภาวะซึมเศร้า,ความเครียด_score,ความเครียด,การดื่มแอล_score,การดื่มแอล,ออกกำลังกาย_score,ออกกำลังกาย
76,2025-06-02,10000036,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,-1,ไม่สูบ,999,999,0,ไม่มีความเสี่ยง,0,แทบไม่มี,3,ดื่มเป็นประจำทุกวัน,0,ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก
113,2025-06-10,10000058,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,1,เป็นบางครั้ง,3,ดื่มเป็นประจำทุกวัน,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
53,2025-06-12,10000025,1,1-2 วันต่อสัปดาห์,-1,ไม่สูบ,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,999,999,0,ไม่ดื่มเลย,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
81,2025-06-12,10000040,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,1,3 - 5 วันต่อสัปดาห์,0,ไม่มีความเสี่ยง,0,แทบไม่มี,1,ดื่ม 1 - 3 วันต่อสัปดาห์,2,ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์
141,2025-06-13,10000076,1,1-2 วันต่อสัปดาห์,0,999,2,6 - 7 วันต่อสัปดาห์,1,มีความเสี่ยง,0,แทบไม่มี,2,ดื่ม 4 - 6 วันต่อสัปดาห์,0,ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก


In [28]:
# TODO: สร้าง Disease Risk Warehouse
# Hint: 1. ดึงค่าจาก memberdata (sys/dias, bmi, waistline, height)
#       2. สร้างฟังก์ชันคำนวณความเสี่ยงทั้ง 3 ประเภท
#       3. Loop แต่ละสมาชิกและคำนวณ
#       4. สร้าง DataFrame จากผลลัพธ์

# เขียนโค้ดที่นี่
# ดึงข้อมูลล่าสุด
latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)
latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)
latest_memberdata = memberdata.sort_values(['memberid', 'name', 'dateadd']).groupby(['memberid', 'name']).tail(1)

# ฟังก์ชันดึงค่าจาก memberdata
def get_value(member_id, name):
    try:
        result = latest_memberdata[(latest_memberdata['memberid'] == member_id) &
                                  (latest_memberdata['name'] == name)]['value']
        if len(result) > 0:
            value = result.iloc[0]
            if pd.notna(value) and str(value) not in ['nan', '- - -', '', 'null']:
                return str(value) if name == 'sys/dias' else float(value)
        return None
    except:
        return None

# แยกความดัน sys/dias
def parse_bp(bp):
    if pd.isna(bp) or str(bp) in ['nan', 'None', '- - -', '']: return None, None
    try:
        parts = str(bp).strip().split('/')
        if len(parts) == 2:
            sys = float(parts[0].strip()) if parts[0].strip().replace('.', '').isdigit() else None
            dias = float(parts[1].strip()) if parts[1].strip().replace('.', '').isdigit() else None
            return sys, dias
    except:
        pass
    return None, None

# ระดับความดัน
def bp_status(sys, dias):
    if pd.isna(sys) or pd.isna(dias): return None
    sys_level = "ปกติ" if 50 < sys < 130 else "สูง" if 130 <= sys < 140 else "ค่อนข้างสูง" if 140 <= sys < 180 else "สูงมาก" if sys >= 180 else None
    dias_level = "ปกติ" if 50 < dias < 85 else "สูง" if 85 <= dias < 90 else "ค่อนข้างสูง" if 90 <= dias < 110 else "สูงมาก" if dias >= 110 else None
    if sys_level == "สูงมาก" or dias_level == "สูงมาก": return "สูงมาก"
    if sys_level == "ค่อนข้างสูง" or dias_level == "ค่อนข้างสูง": return "ค่อนข้างสูง"
    if sys_level == "สูง" or dias_level == "สูง": return "สูง"
    if sys_level == "ปกติ" or dias_level == "ปกติ": return "ปกติ"
    return None

# คะแนนเสี่ยงเบาหวาน (FINDRISC)
def diabetes_risk(age, gender, bmi, waist, sys, dias, rel_dm):
    if pd.isna(age) or pd.isna(gender): return None, None
    score, prob = 0, 0

    # อายุ
    if age < 45: score += 1; prob += 1 * -0.0702134
    elif age < 60: score += 1; prob += 1 * 0.2718858
    else: score += 1; prob += 1 * 0.6043599

    # เพศ
    if gender == 1: score += 2; prob += 2 * 0.4422573

    # BMI
    if not pd.isna(bmi):
        if 23 <= bmi < 27.5: score += 1; prob += 1 * 0.6958621
        elif bmi >= 27.5: score += 1; prob += 1 * 1.235097

    # รอบเอว
    if not pd.isna(waist):
        if (gender == 1 and waist >= 90) or (gender == 0 and waist >= 80):
            score += 1; prob += 1 * 0.5567118

    # ความดัน
    if not pd.isna(sys) and not pd.isna(dias):
        if sys >= 140 or dias >= 90: score += 1; prob += 1 * 0.6409517

    # ญาติเบาหวาน
    if not pd.isna(rel_dm) and rel_dm == 1: score += 1; prob += 1 * 1.081356

    # คำนวณเปอร์เซ็นต์
    exp_val = np.exp(prob + -3.580397)
    percentage = (exp_val / (1 + exp_val)) * 100
    return score, percentage

# ระดับเสี่ยงเบาหวาน
def dm_level(score):
    if pd.isna(score): return None
    if score <= 2: return "ปกติ"
    if score <= 5: return "สูง"
    if score <= 8: return "ค่อนข้างสูง"
    return "สูงมาก"

# คะแนนเสี่ยงหัวใจ-หลอดเลือด
def cardio_risk(age, gender, smoking, dm, sbp, waist, height):
    if any(pd.isna(v) for v in [age, gender, sbp, waist, height]): return None, None
    sex = 1 if gender == 1 else 0
    smoke = 1 if smoking == 1 else 0
    dm_val = 1 if dm == 1 else 0
    score = (0.079 * age) + (0.128 * sex) + (0.019350987 * sbp) + (0.58454 * dm_val) + (3.512566 * (waist / height)) + (0.459 * smoke)
    percentage = (1 - (0.978296 ** np.exp(score - 7.720484))) * 100
    return score, percentage

# ระดับเสี่ยงหัวใจ
def cardio_level(pct):
    if pd.isna(pct): return None
    if pct < 10: return "ปกติ"

# สร้างข้อมูล
risk_records = []
for member_id in latest_memberdata['memberid'].unique():
    # ดึงข้อมูล
    profile = latest_profiles[latest_profiles['memberid'] == member_id]
    age = profile['age'].iloc[0] if len(profile) > 0 else None
    gender = profile['gender'].iloc[0] if len(profile) > 0 else None

    record = latest_records[latest_records['memberid'] == member_id]
    rel_dm = record['relatives_diabetes'].iloc[0] if len(record) > 0 else None
    rec_dm = record['record_diabetes'].iloc[0] if len(record) > 0 else None

    habit = latest_habits[latest_habits['memberid'] == member_id]
    smoking = habit['smoking'].iloc[0] if len(habit) > 0 else None

    # ดึงค่าการวัด
    sys_dias = get_value(member_id, 'sys/dias')
    bmi = get_value(member_id, 'bmi')
    waist = get_value(member_id, 'waistline')
    height = get_value(member_id, 'height')

    sys, dias = parse_bp(sys_dias)

    # 1. ความดันโลหิต
    bp_stat = bp_status(sys, dias)
    bp_score = f"{int(sys) if sys else ''}/{int(dias) if dias else ''}" if sys or dias else 999
    risk_records.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'โรคความดันโลหิตสูง',
        'ระดับความเสี่ยง': bp_stat if bp_stat else "ไม่ระบุ",
        'คะแนน': bp_score
    })

    # 2. เบาหวาน
    dm_score, dm_pct = diabetes_risk(age, gender, bmi, waist, sys, dias, rel_dm)
    dm_lv = dm_level(dm_score) if dm_score else "ไม่ระบุ"
    dm_txt = f"{dm_score}/{dm_pct:.2f}" if dm_score and dm_pct else 999
    risk_records.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'เสี่ยงโรคเบาหวาน',
        'ระดับความเสี่ยง': dm_lv,
        'คะแนน': dm_txt
    })

    # 3. หัวใจ-หลอดเลือด
    cv_score, cv_pct = None, None
    cv_lv = None
    cv_txt = 999
    if all(v is not None for v in [age, gender, sys, waist, height]):
        cv_score, cv_pct = cardio_risk(age, gender, smoking, rec_dm, sys, waist, height)
        if cv_pct:
            cv_lv = cardio_level(cv_pct)
            cv_txt = f"{cv_pct:.2f}%"

    risk_records.append({
        'member_id': member_id,
        'ภาวะเสี่ยง': 'เสี่ยงโรคหัวใจและหลอดเลือด',
        'ระดับความเสี่ยง': cv_lv if cv_lv else "ไม่ระบุ",
        'คะแนน': cv_txt
    })

disease_risk_wh = pd.DataFrame(risk_records)
print(f"✅ Disease Risk Warehouse: {len(disease_risk_wh)} แถว")
disease_risk_wh.head(15)

✅ Disease Risk Warehouse: 300 แถว


,member_id,ภาวะเสี่ยง,ระดับความเสี่ยง,คะแนน
0,10000000,โรคความดันโลหิตสูง,ปกติ,120/71
1,10000000,เสี่ยงโรคเบาหวาน,สูง,4/23.35
2,10000000,เสี่ยงโรคหัวใจและหลอดเลือด,ปกติ,5.27%
3,10000001,โรคความดันโลหิตสูง,ค่อนข้างสูง,144/78
4,10000001,เสี่ยงโรคเบาหวาน,สูง,4/19.54
5,10000001,เสี่ยงโรคหัวใจและหลอดเลือด,ปกติ,7.07%
6,10000002,โรคความดันโลหิตสูง,ค่อนข้างสูง,116/93
7,10000002,เสี่ยงโรคเบาหวาน,สูง,4/29.41
8,10000002,เสี่ยงโรคหัวใจและหลอดเลือด,ปกติ,5.97%
9,10000003,โรคความดันโลหิตสูง,ค่อนข้างสูง,124/100


### Exercise 3.5: Disease Risk Warehouse

**โจทย์**: สร้างตารางความเสี่ยงโรค 3 ประเภท

**ความเสี่ยงที่ต้องคำนวณ**:
1. **โรคความดันโลหิตสูง**: จากค่า sys/dias
2. **เสี่ยงโรคเบาหวาน**: ใช้ FINDRISC algorithm (คำนวณจาก age, gender, BMI, waistline, sys/dias, relatives_diabetes)
3. **เสี่ยงโรคหัวใจและหลอดเลือด**: คำนวณจาก age, gender, smoking, diabetes, sys, waist/height ratio

**โครงสร้างข้อมูล**:
- แต่ละสมาชิกจะมี 3 แถว (1 แถวต่อ 1 ประเภทความเสี่ยง)
- คอลัมน์: member_id, ภาวะเสี่ยง, ระดับความเสี่ยง, คะแนน

**Hint**: ดูตัวอย่างจาก detailed_health_warehouse_tutorial.ipynb

---

## 📤 Part 4: อัปโหลดไปยัง Google Sheets (15 นาที)

### Exercise 4.1: Authentication

**โจทย์**: Authenticate กับ Google เพื่อใช้งาน Google Sheets API

In [24]:
# TODO: Authenticate กับ Google
# Hint: ใช้ from google.colab import auth
#       auth.authenticate_user()
#       แล้ว authorize กับ gspread

# เขียนโค้ดที่นี่
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

### Exercise 4.3: อัปโหลดข้อมูล

**โจทย์**: อัปโหลดข้อมูลทั้ง 5 warehouses ไปยัง Google Sheets โดยแต่ละ warehouse เป็น 1 sheet

**ชื่อ Sheet**:
- 01-ข้อมูลผู้ตรวจสุขภาพ
- 02-ประวัติโรคNCDs
- 03-พฤติกรรมการกิน
- 04-พฤติกรรมรวม
- 05-ความเสี่ยงโรค

In [27]:
def upload_to_sheet(spreadsheet, df, sheet_name):
    """อัปโหลด DataFrame ไปยัง Google Sheet"""
    try:
        # หา/สร้าง worksheet
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(
            title=sheet_name,
            rows=str(max(len(df), 1)),
            cols=str(max(len(df.columns), 1))
        )

    # ล้างข้อมูลเดิม
    worksheet.clear()

    # แปลง DataFrame เป็น list (ระวัง date objects!)
    data_to_upload = [df.columns.values.tolist()] + df.astype(str).values.tolist()

    # อัปโหลด
    worksheet.update(data_to_upload)
    print(f"✅ อัปโหลด '{sheet_name}' สำเร็จ ({len(df)} แถว)")


# ---------------------------------------------------------------------
# TODO: อัปโหลดทั้ง 5 warehouses และสร้าง URL อัตโนมัติ
# ---------------------------------------------------------------------

HEALTH_DW_SHEET_NAME = 'Health_Data_Warehouse_Lab'  # <-- แก้ชื่อไฟล์ของคุณที่นี่

try:
    # เปิด Google Sheets ไฟล์
    spreadsheet = gc.open(HEALTH_DW_SHEET_NAME)
    print(f"✅ เปิด Google Sheets '{HEALTH_DW_SHEET_NAME}' สำเร็จ")
except gspread.SpreadsheetNotFound:
    print(f"⚠️ ไม่พบ Google Sheets '{HEALTH_DW_SHEET_NAME}'. กำลังสร้างไฟล์ใหม่...")
    spreadsheet = gc.create(HEALTH_DW_SHEET_NAME)
    print(f"✅ สร้าง Google Sheets '{HEALTH_DW_SHEET_NAME}' สำเร็จ")

# ✅ สร้าง URL ของ Sheet อัตโนมัติ
sheet_id = spreadsheet.id
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit"
print(f"🔗 URL ของ Google Sheet คือ:\n{sheet_url}\n")

# อัปโหลดแต่ละ warehouse
upload_to_sheet(spreadsheet, health_profile_wh, '01-ข้อมูลผู้ตรวจสุขภาพ')
upload_to_sheet(spreadsheet, ncds_disease_wh, '02-ประวัติโรคNCDs')
upload_to_sheet(spreadsheet, eating_behavior_wh, '03-พฤติกรรมการกิน')
upload_to_sheet(spreadsheet, comprehensive_behavior_wh, '04-พฤติกรรมรวม')
# upload_to_sheet(spreadsheet, disease_risk_wh, '05-ความเสี่ยงโรค')  # ถ้ายังไม่มีให้ปิดไว้ก่อน

print("\n🎉 อัปโหลด Data Warehouse ทั้งหมดเสร็จสิ้น!")


✅ เปิด Google Sheets 'Health_Data_Warehouse_Lab' สำเร็จ
🔗 URL ของ Google Sheet คือ:
https://docs.google.com/spreadsheets/d/1V69GbFjO7yiSoNYImBNSTI5FOP8839OetMkL9rnS1-w/edit

✅ อัปโหลด '01-ข้อมูลผู้ตรวจสุขภาพ' สำเร็จ (100 แถว)
✅ อัปโหลด '02-ประวัติโรคNCDs' สำเร็จ (308 แถว)
✅ อัปโหลด '03-พฤติกรรมการกิน' สำเร็จ (100 แถว)
✅ อัปโหลด '04-พฤติกรรมรวม' สำเร็จ (100 แถว)

🎉 อัปโหลด Data Warehouse ทั้งหมดเสร็จสิ้น!


---

## 📝 Part 5: สรุปและคำถาม (10 นาที)

### คำถามท้ายบท:

**Q1**: ทำไมต้องดึงข้อมูล "ล่าสุด" ของแต่ละสมาชิก ไม่ใช่ข้อมูลทั้งหมด?

**คำตอบ**: _______________________________________________________

---

**Q2**: ในตาราง NCDs Disease ทำไมถึงมีหลายแถวต่อ 1 สมาชิก?

**คำตอบ**: _______________________________________________________

---

**Q3**: อะไรคือความแตกต่างระหว่าง "_score" และ "label" ในตาราง Eating Behavior?

**คำตอบ**: _______________________________________________________

---

**Q4**: ลองคิดว่าถ้าจะเพิ่ม Warehouse ตารางที่ 6 ควรจะเป็นตารางอะไร? และมีข้อมูลอะไรบ้าง?

**คำตอบ**: _______________________________________________________

---

## ✅ Checklist สำหรับการส่งงาน

- [ ] สร้าง Health Profile Warehouse ได้อย่างน้อย 100 แถว
- [ ] สร้าง NCDs Disease Warehouse ได้
- [ ] สร้าง Eating Behavior Warehouse ได้
- [ ] สร้าง Comprehensive Behavior Warehouse ได้
- [ ] สร้าง Disease Risk Warehouse ได้
- [ ] อัปโหลดข้อมูลไปยัง Google Sheets สำเร็จ
- [ ] ตอบคำถามท้ายบทครบทั้ง 4 ข้อ
- [ ] แชร์ Google Sheets ให้อาจารย์ดูได้

## 📤 วิธีการส่งงาน

1. Save notebook นี้ (Download .ipynb)
2. Copy URL ของ Google Sheets ที่สร้าง
3. ส่งทั้ง 2 อย่างใน Google Classroom

---

**Google Sheets URL**: _______________________________________________________

**หมายเหตุ**: กรุณาแชร์ Google Sheets ให้อาจารย์สามารถดูได้ (Viewer)

---

## 🎓 เกณฑ์การให้คะแนน (100 คะแนน)

| หัวข้อ | คะแนน |
|--------|-------|
| Part 1-2: เตรียมสภาพแวดล้อมและดึงข้อมูล | 15 |
| Part 3.1: Health Profile Warehouse | 20 |
| Part 3.2: NCDs Disease Warehouse | 12 |
| Part 3.3: Eating Behavior Warehouse | 12 |
| Part 3.4: Comprehensive Behavior Warehouse | 12 |
| Part 3.5: Disease Risk Warehouse | 19 |
| Part 4: อัปโหลดไปยัง Google Sheets | 5 |
| Part 5: คำถามท้ายบท | 5 |
| **รวม** | **100** |

---

**สิ่งที่ได้เรียนรู้**:
✅ การทำ Data Transformation  
✅ การใช้ Pandas สำหรับ Data Wrangling  
✅ การสร้าง Data Warehouse  
✅ การคำนวณความเสี่ยงโรค (FINDRISC, Cardiovascular Risk)  
✅ การใช้ Google Sheets API  

**Good Luck! 🎓**

In [ ]:
# ปิดการเชื่อมต่อ Database
conn.close()
print("✅ ปิดการเชื่อมต่อ Database เรียบร้อยแล้ว")